# Predicting Heart Disease Using Machine Learning

This notebook explores heart disease data using machine learning through several Python packages. 
The goal is to create a model that can predict whether a patient has heart disease based on other features.

**Steps:**
1. Problem Definition
2. Data
3. Evaluation
4. Feature Selection
5. Model Creation
6. Experimentation

*Note: Data is derived from the UCI ML Repository Website*

[Kaggle Link](https://www.kaggle.com/ronitf/heart-disease-uci)

**Attribute Information:**
1. age
2. sex
3. chest pain type (4 values)
4. resting blood pressure
5. serum cholestoral in mg/dl
6. fasting blood sugar > 120 mg/dl
7. resting electrocardiographic results (values 0,1,2)
8. maximum heart rate achieved
9. exercise induced angina
10. oldpeak = ST depression induced by exercise relative to rest
11. the slope of the peak exercise ST segment
12. number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect